Download dataset

In [ ]:
!mkdir data
!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/labels.txt
!wget -c https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/reviews.txt
!mv *.txt data/

--2021-12-17 09:15:50--  https://github.com/agungsantoso/deep-learning-v2-pytorch/raw/master/sentiment-rnn/data/labels.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/agungsantoso/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt [following]
--2021-12-17 09:15:51--  https://raw.githubusercontent.com/agungsantoso/deep-learning-v2-pytorch/master/sentiment-rnn/data/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.007s  

2021-12-17 09:15:51 (30.5 MB/

In [ ]:
import numpy as np

# read data from text files
with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

In [ ]:
print(reviews[:1000])
print()
print(labels[:30])

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

As data preprocessing, we remove all punctuation, then get all the text without the newlines and split it into individual words.

In [ ]:
from string import punctuation

print(punctuation)

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# split by new lines and spaces
reviews_split = all_text.split('\n')
all_text = ' '.join(reviews_split)

# create a list of words
words = all_text.split()

Check the first 20 words

In [ ]:
words[:20]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such']

You need to encoding the text into integers to be used as input to your neural network model. To pad 0s to make each sequence length the same, those integers start from 1, not 0. You may build a dictionary to map words to integers.

In [ ]:
# feel free to use this import
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab,1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
  reviews_ints.append([vocab_to_int[word] for word in review.split()])

Test if it works fine

In [ ]:
# stats about vocabulary
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()

# print tokens in first review
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  74072

Tokenized review: 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


Encoding the labels to 0 and 1

In [ ]:
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
encoded_labels = np.array([1 if label == 'positive' else 0 for label in labels_split])

Check if GPU is available for training

In [ ]:
import torch

# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

No GPU available, training on CPU.


TO DO: Remove the seqeunces with 0 length.

In [ ]:
for i in range(len(reviews_ints)):
  if len(reviews_ints[i]) == 0:
    reviews_ints.remove(reviews_ints[i])
    print(i)

encoded_labels = np.array(encoded_labels)
encoded_labels = encoded_labels[:25000]

25000


TO DO: Create padded sequences with the equal length, e.g., 100. You can choose the sequence length here. If the sequence length is larger than your choosen length, you will truncate it so that all sequences have the same length (after 0 padding for shorter sequences).

In [ ]:
from copy import deepcopy
reviews_ints_copy = deepcopy(reviews_ints)
padding = 100

for i in range(len(reviews_ints_copy)):
  if len(reviews_ints_copy[i]) < padding:
    count = len(reviews_ints_copy[i])
    for j in range(padding - count):
      reviews_ints_copy[i] = np.append(reviews_ints_copy[i], 0)
  elif len(reviews_ints_copy[i]) > padding:
    reviews_ints_copy[i] = reviews_ints_copy[i][:padding]

reviews_ints_copy = np.vstack(reviews_ints_copy)



TO DO: Create your own dataset class definition (inherited from torch.utils.data.Dataset) and use DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader

class Sentiment(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        X = self.X[index]
        Y = self.Y[index]
        return X, Y

    def __len__(self):
        return len(self.X)


training_data = Sentiment(reviews_ints_copy[:20000], encoded_labels[:20000])
test_data = Sentiment(reviews_ints_copy[20000:], encoded_labels[20000:])

train_loader = DataLoader(training_data, batch_size=500, shuffle=True)
test_loader = DataLoader(test_data, batch_size=500, shuffle=True)

TO DO: Create your own RNN model class definition (inherited from torch.nn.Module)

In [ ]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)

        return self.fc(hidden.squeeze(0))

TO DO: Create your code for training the model

TO DO: Create your code for testing (evaluation on test set)


In [ ]:
from sklearn.preprocessing import binarize

model = RNN(len(vocab_to_int)+1, 256, 1024, 1)
model.eval()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 100

for epoch in range(epochs):
  for data, label in train_loader:

    output = model(data.T)
    loss = criterion(output.flatten(), label.flatten().type(torch.FloatTensor))
    print("Training loss : {}".format(loss))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  hit = 0
  tot = 0
  with torch.no_grad():
    for data, label in test_loader:

      outputs = model(data.T)
      output = outputs.flatten().detach().cpu().numpy()
      output_n = binarize(output.reshape(-1,1), threshold=0)
      labels_n = label.flatten().detach().cpu().numpy()

      for i in range(output_n.shape[0]):
        if output_n[i] == labels_n[i]:
          hit += 1
          tot += 1
        else:
          tot += 1
    print("===========================================================")
    print("Test Acc on Epoch {}".format(epoch) + ": {}".format(hit/tot))
    print("===========================================================")




Training loss : 0.6938515901565552
Training loss : 0.7019339799880981
Training loss : 0.7047690153121948
Training loss : 0.6985001564025879
Training loss : 0.6951850056648254
Training loss : 0.6979073286056519
Training loss : 0.6938180923461914
Training loss : 0.693930983543396
Training loss : 0.6901216506958008
Training loss : 0.6895695924758911
Training loss : 0.6981564164161682
Training loss : 0.700493574142456
Training loss : 0.6887861490249634
Training loss : 0.7025400996208191
Training loss : 0.6959770321846008
Training loss : 0.6928430199623108
Training loss : 0.6883878111839294
Training loss : 0.6989741325378418
Training loss : 0.6994673013687134
Training loss : 0.6907265186309814
Training loss : 0.6975096464157104
Training loss : 0.7014065980911255
Training loss : 0.6908881664276123
Training loss : 0.6961145997047424
Training loss : 0.6893723607063293
Training loss : 0.6976418495178223
Training loss : 0.693834125995636
Training loss : 0.6960244178771973
Training loss : 0.69758

KeyboardInterrupt: ignored

===========================================================
Test Acc on Epoch 13: 0.6954
===========================================================